In [ ]:
import win32com.client
import pythoncom
import time
import os

def Plotear_A1():

    # PARA CONECTAR CON AUTOCAD
    acad = win32com.client.Dispatch("AutoCAD.Application")
    doc = acad.ActiveDocument

    # NOMBRE DEL ARCHIVO 
    nombre_dwg = doc.Name
    nom = nombre_dwg[:-4]   

    # RUTA DE SALIDA 
    ruta = r"C:\Users\....\....\...." # <<< AQUI DEBEN PONER LA DIRECCION DE LA CARPETA DONDE GUARDARAN LOS PDFS
    os.makedirs(ruta, exist_ok=True)

    # FORZAR LAYOUT DE PAPEL 
    doc.ActiveLayout = doc.Layouts.Item("Layout1") # <<< AQUI DEBEN PONER EL NOMBRE DEL LAYOUT, POR DEFECTO "Layout1"
    layout = doc.ActiveLayout

    #  LIMPIAR SELECTION SET 
    try:
        doc.SelectionSets.Item("SS1").Delete()
    except:
        pass

    ss = doc.SelectionSets.Add("SS1")
    ss.SelectOnScreen()

    contador = 1

    for ent in ss:

        if ent.EntityName != "AcDbPolyline":
            continue

        #  BOUNDING BOX 
        coords = ent.Coordinates

        x = coords[0::2]
        y = coords[1::2]

        xmin, xmax = min(x), max(x)
        ymin, ymax = min(y), max(y)

        P1 = win32com.client.VARIANT(
            pythoncom.VT_ARRAY | pythoncom.VT_R8, (xmin, ymin)
        )
        P2 = win32com.client.VARIANT(
            pythoncom.VT_ARRAY | pythoncom.VT_R8, (xmax, ymax)
        )

        # ARCHIVO PDF 
        plotfile = os.path.join(ruta, f"{nom}-{contador}.pdf")

        # CONFIGURACIÓN PARA EL PLOTEO 
        layout.ConfigName = "DWG To PDF.pc3"
        layout.CanonicalMediaName = "ISO_full_bleed_A1_(841.00_x_594.00_MM)"
        layout.PaperUnits = 1
        layout.PlotRotation = 0
        layout.StyleSheet = "None" # <<< AQUI DEBEN PONER SU "PLOT STYLE TABLE" (SI LO TIENEN)

        layout.PlotType = 4  # acWindow
        layout.SetWindowToPlot(P1, P2)

        layout.UseStandardScale = False
        layout.SetCustomScale(1, 1)
        layout.StandardScale = 0

        # REFRESCAR AUTOCAD 
        doc.SetVariable("BACKGROUNDPLOT", 0)
        doc.SetVariable("PLOTOUTPUTOPEN", 0)  # <<< ES PARA EVITAR QUE SE ABRAN LOS PDF, SI QUIEREN QUE SE ABRAN COMO EN EL VIDEO LE PONEN "1"
        doc.Regen(1)
        time.sleep(0.3)

        # CENTRAR Y PLOTEAR 
        layout.CenterPlot = True
        doc.Plot.PlotToFile(plotfile)

        contador += 1
        time.sleep(1)

    print("✔ PDFs generados correctamente")

def main():
    Plotear_A1()

main()
